In [16]:
using CSV
using DataFrames
using Base.Threads
using Statistics

In [17]:
# Paso 1: Leer el archivo CSV
df_movies = CSV.read("C:/Users/felip/Desktop/Julia/movies.csv", DataFrame)
df_ratings = CSV.read("C:/Users/felip/Desktop/Julia/ratings.csv", DataFrame)
"Done"

"Done"

In [18]:
# Paso 2: Dividir el DataFrame en 8 partes
num_partes = 8
partes = [df_ratings[(i-1)*div(end, num_partes)+1:min(i*div(end, num_partes), end), :] for i in 1:num_partes]
"Done"

"Done"

In [19]:
#Definimos una función que haga un registro por cada género en una película
function movies_exploded_function(df_movies)
    df_movies.genre_list = map(x -> split(x, "|"), df_movies.genres)
    convert(Vector{Vector{String}}, df_movies.genre_list)
    lista_genero = []
    lista_movieId = []
    for i  in 1:size(df_movies,1)
        for j in 1:size(df_movies.genre_list[i:i][1],1)
            push!(lista_genero, df_movies.genre_list[i:i][1][j])
            push!(lista_movieId, df_movies.movieId[i:i][1])
        end
    end
    movies_exploded = DataFrame(movieId = lista_movieId, genre = lista_genero)
    return movies_exploded
end

movies_exploded_function (generic function with 1 method)

In [20]:
movies_exploded = movies_exploded_function(df_movies)
"Done"

"Done"

In [21]:
function ratings_per_genre_function(parte)
    movies_ratings = innerjoin(movies_exploded, parte, on=:movieId)
    ratings_per_genre = combine(groupby(movies_ratings, :genre), nrow => :rating_count, :rating => mean)
    #ruta_csv = "C:\\Users\\PC\\Desktop\\DiplomadoIA\\Julia\\parte_{}.csv"
    #CSV.write(ruta_csv, df)
    return ratings_per_genre
end

ratings_per_genre_function (generic function with 1 method)

In [22]:
# Función de procesamiento
function procesar(parte)
    ratings_per_genre_function(parte)
end

procesar (generic function with 1 method)

In [23]:
# Paso 3: Procesar en multihilos
resultados = Vector{Any}(undef, num_partes)
Threads.@threads for i in 1:num_partes
    resultados[i] = procesar(partes[i])
end

# Mostrar resultados
println(resultados)

Any[20×3 DataFrame
 Row │ genre               rating_count  rating_mean 
     │ Any                 Int64         Float64     
─────┼───────────────────────────────────────────────
   1 │ Adventure                1013433      3.52717
   2 │ Animation                 288363      3.61783
   3 │ Children                  360205      3.43948
   4 │ Comedy                   1480335      3.43585
   5 │ Fantasy                   489383      3.51502
   6 │ Action                   1290266      3.48191
   7 │ Drama                    1844496      3.68893
   8 │ War                       213418      3.79486
   9 │ Sci-Fi                    762116      3.49705
  10 │ Romance                   732387      3.54659
  11 │ Musical                   154602      3.54506
  12 │ Crime                     717000      3.69839
  13 │ Thriller                 1155282      3.5362
  14 │ Horror                    326520      3.31884
  15 │ Mystery                   345848      3.6749
  16 │ IMAX               

In [26]:
combined_df = resultados[1]
for i in 2:num_partes
    combined_df = vcat(combined_df, resultados[i])
end

In [25]:
combined_df = combine(groupby(combined_df, :genre), :rating_count => sum, :rating_mean => mean)

Row,genre,rating_count_sum,rating_mean_mean
,Any,Int64,Float64
1,Adventure,8048929,3.52599
2,Animation,2329566,3.61555
3,Children,2897960,3.43772
4,Comedy,11860141,3.43222
5,Fantasy,3899930,3.51433
6,Action,10222143,3.47996
7,Drama,14787039,3.68585
8,War,1697649,3.79759
9,Sci-Fi,6032230,3.49435
